In [58]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [59]:
#path_ref = '../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'
path_ref = '../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL'
path_ref_name = 'xc7a100t-1csg324-Artix-7-200-MHz-KERNEL-DECIMAL-'

listD = ["Bambu"]
# listD = ["Arm","Atmega328p","Leon3","RiscV","Thumb"] # Insert the device/ISS you want to analyze
# listD = ["Bambu"]
# listDT = ["int8_t","int16_t","int32_t","int64_t"] # Insert the data type you want to analyze
listDT = ["float","double"]

In [60]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "01-00-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [61]:
%%log_time 1

# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()

for root, dirs, files in os.walk(path_ref): # Change folder
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)

nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+str(nameFileS))
print('pathFileS:'+str(pathFileS))
print('rootFileS:'+str(rootFileS))

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

nameFileS:0    parameters.json
1    parameters.json
2    parameters.json
3    parameters.json
4    parameters.json
5    parameters.json
6    parameters.json
7    parameters.json
8    parameters.json
dtype: object
pathFileS:0               ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\astar\parameters.json
1    ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\banker_algorithm\parameters.json
2         ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\bellmanford\parameters.json
3                 ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\bfs\parameters.json
4       ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\floydwarshall\parameters.json
5                 ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\gcd\parameters.json
6             ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\kruskal\parameters.json
7           ../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KE

In [62]:
%%log_time 2

# .json file loop
#df = pd.DataFrame(columns=['Function','Device','Data_Type',
#                           'Num_Sc','Range_Sc','Num_ScI','Range_ScI',
#                           'Num_Arr','Range_Arr'])

listrange = [0, 0]
listrangeI = [0, 0]
listdtI = [' ']
listdt = [' ']
listfunI = [' ']
listfun = [' ']
listdevI = [' ']
listdev = [' ']
listboardI = [' ']
listboard = [' ']

for i in range(0, len(pathFile)):
    pF = pathFile[i]
    print('pF:'+str(pF))

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        # spR = rootFile[i].split('\\')
        spR = rootFile[i].rsplit('\\', 1)
        print('spR:'+str(spR))
    else:
        # spR = rootFile[i].split('/')
        spR = rootFile[i].rsplit('/', 1)
        print('spR:'+str(spR))
    
    function = spR[-1]
    print('function:'+str(function))
    
    for j in range(0, len(listD)):
        board = listD[j]
        print('board:'+str(board))

        #Device loop
        deviceList = list(obj.keys())
        print('deviceList:'+str(deviceList))
        deviceListR = []
        for u in range(0, len(deviceList)):
            if deviceList[u] in listD:
                deviceListR.append(deviceList[u])
                print('deviceListR:'+str(deviceList[u]))

        for n in range(0, len(deviceListR)):
            # Data_type loop
            device = deviceListR[n]
            print('device:'+str(device))
            dtarr = list(obj[deviceListR[n]].keys())
            print('dtarr:'+str(dtarr))

            dtarrR = []
            for u in range(0, len(dtarr)):
                if dtarr[u] in listDT:
                    dtarrR.append(dtarr[u])
                    print(f'dtarr[{u}]:'+str(dtarr[u]))

            for j in range(0, len(dtarrR)):
                dataype = dtarrR[j]
                print('dataype:'+str(dtarrR[j]))
                lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
                print('lsubNP:'+str(lsubNP))
                lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))
                print('lsubVP:'+str(lsubVP))
                ltot = len(lsubNP)
                print('ltot:'+str(ltot))

                kl = []
                for k in range(0, ltot):
                    flag = 0
                    if not "[" in lsubNP[k]:
                        for keysl in lsubNP:
                            if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                                flag = 1
                                kl.append(k) 
                                print('kl:'+str(k))

                m = 0
                for item in lsubVP:
                    #print(function, lsubVP)
                    if ';' in item:
                        newstr = item.split(";")[0]
                        newstr = newstr.replace("[", "")
                        newstr = newstr.replace("]", "")
                        try:
                            el1 = float(newstr.split(",")[0])
                            el2 = float(newstr.split(",")[1])
                        except ValueError as e:
                            print(f"ValueError: could not convert string to float: {e}")
                            el1 = 0
                            el2 = 0
                        strint = el1
                        rangeint = el2 - el1
                        lr = np.array([strint, rangeint])
                        if m in kl:
                            listrangeI = np.vstack([listrangeI, lr])
                            listdtI.append(dataype)
                            listfunI.append(function)
                            # listboardI.append(board)
                            listdevI.append(device)
                        else:
                            listrange = np.vstack([listrange, lr])
                            listdt.append(dataype)
                            listfun.append(function)
                            # listboard.append(board)
                            listdev.append(device)
                    m = m + 1

pF:../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\astar\parameters.json
spR:['../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL', 'astar']
function:astar
board:Bambu
deviceList:['args', 'ALL', 'Arm', 'Thumb', 'Bambu', 'Atmega328p', '8051', 'Leon3', 'RiscV']
deviceListR:Bambu
device:Bambu
dtarr:['int8_t', 'uint8_t', 'int16_t', 'uint16_t', 'int32_t', 'uint32_t', 'int64_t', 'uint64_t', 'float', 'double']
dtarr[8]:float
dtarr[9]:double
dataype:float
lsubNP:['size', 'a[size][size]', 'goal']
lsubVP:['[2,32];10', '[-2147483640,2147483640]', '[2,32];10']
ltot:3
kl:0
dataype:double
lsubNP:['size', 'a[size][size]', 'goal']
lsubVP:['[2,32];10', '[-9223372036854775808,9223372036854775807]', '[2,32];10']
ltot:3
kl:0
pF:../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL\banker_algorithm\parameters.json
spR:['../../../SLIDE-x-D004-Artix-7-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL', 'banker_algorithm']
function:banker_algorithm
board:Bambu
deviceList:['args', 'ALL',

In [63]:
%%log_time 3

listrangeS = pd.DataFrame(listrange)
listdtS = pd.Series(listdt)
listfunS = pd.Series(listfun)
listdevS = pd.Series(listdev)
# listboardS = pd.Series(listboard)
dfS = pd.concat([listdevS, listfunS, listdtS, listrangeS], axis=1) # listboardS, 
dfS.columns = ['DEVICE', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE']
dfS = dfS.iloc[1:]

In [64]:
dfS

,DEVICE,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,Bambu,astar,float,2.0,3.000000e+01
2,Bambu,astar,double,2.0,3.000000e+01
3,Bambu,gcd,float,2.0,2.147484e+09
4,Bambu,gcd,float,2.0,2.147484e+09
5,Bambu,gcd,double,2.0,9.223372e+18
6,Bambu,gcd,double,2.0,9.223372e+18


In [65]:
%%log_time 4

listrangeIS = pd.DataFrame(listrangeI)
listdtIS = pd.Series(listdtI)
listfunIS = pd.Series(listfunI)
listdevIS = pd.Series(listdevI)
# listboardIS = pd.Series(listboardI)
dfSI = pd.concat([listdevIS, listfunIS, listdtIS, listrangeIS], axis=1) # listboardIS ,
dfSI.columns = ['DEVICE', 'FUNCTION', 'DATA_TYPE', 'DATA_START', 'DATA_RANGE'] # 'BOARD',
dfSI = dfSI.iloc[1:]

In [66]:
dfSI

,DEVICE,FUNCTION,DATA_TYPE,DATA_START,DATA_RANGE
1,Bambu,astar,float,2.0,30.0
2,Bambu,astar,double,2.0,30.0
3,Bambu,banker_algorithm,float,2.0,30.0
4,Bambu,banker_algorithm,double,2.0,30.0
5,Bambu,bellmanford,float,2.0,30.0
6,Bambu,bellmanford,double,2.0,30.0
7,Bambu,bfs,float,2.0,30.0
8,Bambu,bfs,double,2.0,30.0
9,Bambu,floydwarshall,float,2.0,30.0
10,Bambu,floydwarshall,double,2.0,30.0


In [67]:
%%log_time 5

serS = dfS.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serS)):
    arrlimt = []
    ml = 0
    w = (serS.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfS.iterrows():
    clf = 0
    for p in range(0, len(serS)):
        if row['DATA_TYPE'] == serS.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfS['RANGE_S'] = clf 

In [68]:
%%log_time 6

serSI = dfSI.groupby(['DATA_TYPE'])['DATA_RANGE'].max()

n = 3
ws = 10
arrlM = []
for d in range(0, len(serSI)):
    arrlimt = []
    ml = 0
    w = (serSI.iloc[d] - ml) // n
    for h in range(n):
        arrlimt.append(ml + h*w)
    arrlM.append(arrlimt)
    
for index, row in dfSI.iterrows():
    clf = 0
    for p in range(0, len(serSI)):
        if row['DATA_TYPE'] == serSI.index[p]:
            ind = p
            arrl = arrlM[ind]
            for u in range(0, len(arrl)-1):
                if (row['DATA_RANGE'] > arrl[u]) and (row['DATA_RANGE'] <= arrl[u+1]):
                    clf = u + 1
                elif (row['DATA_RANGE'] > arrl[u+1]):
                    clf = len(arrl)            
    dfSI['RANGE_SI'] = clf 

In [69]:
dfS.to_csv("dfS_tot.csv", sep=';')
dfSI.to_csv("dfSI_tot.csv", sep=';')

In [70]:
dfS.to_csv("dfS.csv", sep=';')
dfSI.to_csv("dfSI.csv", sep=';')